In [4]:
import pandas as pd
import numpy as np
from sklearn import preprocessing

In [5]:
data = pd.read_csv("data/KaggleV2-May-2016.csv")

In [6]:
data

,PatientId,AppointmentID,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received,No-show
0,2.987250e+13,5642903,F,2016-04-29T18:38:08Z,2016-04-29T00:00:00Z,62,JARDIM DA PENHA,0,1,0,0,0,0,No
1,5.589978e+14,5642503,M,2016-04-29T16:08:27Z,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,0,0,0,0,0,No
2,4.262962e+12,5642549,F,2016-04-29T16:19:04Z,2016-04-29T00:00:00Z,62,MATA DA PRAIA,0,0,0,0,0,0,No
3,8.679512e+11,5642828,F,2016-04-29T17:29:31Z,2016-04-29T00:00:00Z,8,PONTAL DE CAMBURI,0,0,0,0,0,0,No
4,8.841186e+12,5642494,F,2016-04-29T16:07:23Z,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,1,1,0,0,0,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110522,2.572134e+12,5651768,F,2016-05-03T09:15:35Z,2016-06-07T00:00:00Z,56,MARIA ORTIZ,0,0,0,0,0,1,No
110523,3.596266e+12,5650093,F,2016-05-03T07:27:33Z,2016-06-07T00:00:00Z,51,MARIA ORTIZ,0,0,0,0,0,1,No
110524,1.557663e+13,5630692,F,2016-04-27T16:03:52Z,2016-06-07T00:00:00Z,21,MARIA ORTIZ,0,0,0,0,0,1,No
110525,9.213493e+13,5630323,F,2016-04-27T15:09:23Z,2016-06-07T00:00:00Z,38,MARIA ORTIZ,0,0,0,0,0,1,No


### Observations

In [79]:
len(data[data["Age"] == 0]),len(data[data["Age"] < 0]), len(data)

(3539, 1, 110527)

--> some irregularities in age : when unknown, =0?

### Data cleaning

##### Awaiting Days

In [7]:
data.ScheduledDay = pd.to_datetime(data.ScheduledDay)
data.AppointmentDay = pd.to_datetime(data.AppointmentDay)

In [8]:
data[data.ScheduledDay > data.AppointmentDay]
data['AppointmentDay'] = data['AppointmentDay'] + pd.Timedelta('1d') - pd.Timedelta('1s')
#pb for same day registration

In [9]:
data['AwaitingTime'] = (data.AppointmentDay - data.ScheduledDay).dt.days

##### Missed appointments

In [10]:
len(pd.unique(data.PatientId))

62299

In [11]:
len(data)

110527

In [12]:
mask = data.PatientId%1 != 0
data.loc[mask, "PatientId"] *= 100000

In [13]:
data.PatientId = data.PatientId - min(data.PatientId)

In [14]:
le = preprocessing.LabelEncoder()
trained_le = le.fit(data.PatientId)
data.PatientId = trained_le.transform(data.PatientId)

In [15]:
max(data.PatientId)

62298

In [16]:
# number of previous appointments in May
for i in range(max(data.PatientId)+1):
    mask = data.PatientId == i
    data.loc[mask, "NbPrevious"] = range(sum(mask))

KeyboardInterrupt: 

In [ ]:
data.to_csv("data/df_new.csv", index= False)

In [ ]:
data = pd.read_csv("data/df_new.csv")

In [154]:
data

,Unnamed: 0,PatientId,AppointmentID,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received,NoShow,AwaitingTime,NbPrevious
0,0,30845,5642903,F,2016-04-29 18:38:08+00:00,2016-04-29 23:59:59+00:00,62,JARDIM DA PENHA,0,1,0,0,0,0,0,0,0.0
1,1,55325,5642503,M,2016-04-29 16:08:27+00:00,2016-04-29 23:59:59+00:00,56,JARDIM DA PENHA,0,0,0,0,0,0,0,0,0.0
2,2,15722,5642549,F,2016-04-29 16:19:04+00:00,2016-04-29 23:59:59+00:00,62,MATA DA PRAIA,0,0,0,0,0,0,0,0,0.0
3,3,9018,5642828,F,2016-04-29 17:29:31+00:00,2016-04-29 23:59:59+00:00,8,PONTAL DE CAMBURI,0,0,0,0,0,0,0,0,0.0
4,4,23941,5642494,F,2016-04-29 16:07:23+00:00,2016-04-29 23:59:59+00:00,56,JARDIM DA PENHA,0,1,1,0,0,0,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110522,110522,12800,5651768,F,2016-05-03 09:15:35+00:00,2016-06-07 23:59:59+00:00,56,MARIA ORTIZ,0,0,0,0,0,1,0,35,1.0
110523,110523,14603,5650093,F,2016-05-03 07:27:33+00:00,2016-06-07 23:59:59+00:00,51,MARIA ORTIZ,0,0,0,0,0,1,0,35,3.0
110524,110524,27287,5630692,F,2016-04-27 16:03:52+00:00,2016-06-07 23:59:59+00:00,21,MARIA ORTIZ,0,0,0,0,0,1,0,41,0.0
110525,110525,45960,5630323,F,2016-04-27 15:09:23+00:00,2016-06-07 23:59:59+00:00,38,MARIA ORTIZ,0,0,0,0,0,1,0,41,1.0


# Modifs

In [17]:
data = pd.read_csv("data/df_new.csv")

In [23]:
data = data.iloc[: , 1:]

In [24]:
data

,PatientId,AppointmentID,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received,No-show,AwaitingTime,NbPrevious
0,30845,5642903,F,2016-04-29 18:38:08+00:00,2016-04-29 23:59:59+00:00,62,JARDIM DA PENHA,0,1,0,0,0,0,No,0,0.0
1,55325,5642503,M,2016-04-29 16:08:27+00:00,2016-04-29 23:59:59+00:00,56,JARDIM DA PENHA,0,0,0,0,0,0,No,0,0.0
2,15722,5642549,F,2016-04-29 16:19:04+00:00,2016-04-29 23:59:59+00:00,62,MATA DA PRAIA,0,0,0,0,0,0,No,0,0.0
3,9018,5642828,F,2016-04-29 17:29:31+00:00,2016-04-29 23:59:59+00:00,8,PONTAL DE CAMBURI,0,0,0,0,0,0,No,0,0.0
4,23941,5642494,F,2016-04-29 16:07:23+00:00,2016-04-29 23:59:59+00:00,56,JARDIM DA PENHA,0,1,1,0,0,0,No,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110522,12800,5651768,F,2016-05-03 09:15:35+00:00,2016-06-07 23:59:59+00:00,56,MARIA ORTIZ,0,0,0,0,0,1,No,35,1.0
110523,14603,5650093,F,2016-05-03 07:27:33+00:00,2016-06-07 23:59:59+00:00,51,MARIA ORTIZ,0,0,0,0,0,1,No,35,3.0
110524,27287,5630692,F,2016-04-27 16:03:52+00:00,2016-06-07 23:59:59+00:00,21,MARIA ORTIZ,0,0,0,0,0,1,No,41,0.0
110525,45960,5630323,F,2016-04-27 15:09:23+00:00,2016-06-07 23:59:59+00:00,38,MARIA ORTIZ,0,0,0,0,0,1,No,41,1.0


In [25]:
data = data.rename(columns={"No-show": "NoShow"});

In [26]:
data.loc[(data.NoShow == 'No'),'NoShow']= 0
data.loc[(data.NoShow == 'Yes'),'NoShow']= 1

In [28]:
data.to_csv("data/df_new.csv", index= False)